In [2]:
# Esse arquivo pode ser executado no jupyterlab ou no google colab
# Wesley bispo de brito

Identificar o genêro do nome

1. Usando uma base com 100.000 nomes identificamos a maior frequência do genero
para o nome inputado. 99% de acuracia
2. Se não houver registro do nome, um modelo de regressão logistica identifica se o nome é  masculino ou feminino. 93% de acuracia
3. 96% de chance de acerto desde que o nome seja PT-BR, não testedo com nome estrangeiros.

Melhorias Pendentes.

1. Janela amigavel para uso.
2. Disponibilidade de uso no google colab

Instalações do pip:

In [3]:
#!pip install pandas
#!pip install unidecode
#!pip install tensorflow==2.17.0
#!pip install fuzzywuzzy
#!pip install gdown

In [4]:
# @title Insira um nome { run: "auto" }

import sys
import gdown
import unidecode
import pandas as pd
import tensorflow as tf
from fuzzywuzzy import fuzz
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model



def onde_estamos(): # encontrar o local certo onde o modelo sera baixado
    if 'ipykernel' in sys.modules:
        local = "JupyterLab"
        h5_output = 'ML-genero.h5'
    try:
        import google.colab  # Se importar com sucesso, estamos no Colab
        local = "Google Colab"
        h5_output = '/content/ML-genero.h5'
    except ImportError:
        pass

    return h5_output
#csv de dados
#https://drive.google.com/file/d/1Gczd8DCQU4bKCVjc-aDz5V3SdOL7QdE-/view?usp=drive_link
csv_id = '1Gczd8DCQU4bKCVjc-aDz5V3SdOL7QdE-'
url = f'https://drive.google.com/uc?id={csv_id}'
#h5 do modelo
#https://drive.google.com/file/d/1_KXXdjcDxVs7M3DN68NwVUxMaN99OaRq/view?usp=sharing
h5_id = '1_KXXdjcDxVs7M3DN68NwVUxMaN99OaRq'
h5_url = f'https://drive.google.com/uc?id={h5_id}'
h5_output = onde_estamos()
# Baixando os arquivos
gdown.download(h5_url, h5_output, quiet=False)

# Carregar o modelo salvo

model = load_model(h5_output)
data = pd.read_csv(url)
# Função para prever o gênero de um novo nome
def descobrir_genero(nome):
    tokenizer = Tokenizer(char_level=True)
    seq = tokenizer.texts_to_sequences([nome])
    padded = pad_sequences(seq, maxlen=14)
    prediction = model.predict(padded)
    return 'feminino' if prediction < 0.5 else 'masculino'
# Função para determinar o gênero com maior frequência
def qual_genero(nome):
    entrada = nome.upper()  # maiuscula
    entrada = unidecode.unidecode(entrada)  # removendo acentos
    # Atribuindo nomes parecidos a um DataFrame
    similares = data[data['first_name'].str.contains(entrada, na=False)][['first_name', 'classification', 'ratio', 'frequency_total']]
    # se o nome nao estiver na base:
    # modelo sugere o gênero
    if similares.empty:
        genero = descobrir_genero(nome)
        print(f'Para o nome {nome}, o modelo sugere que seja {genero}.')
        return
    # Calculando similaridade
    def calculate_similarity(name):
        return fuzz.ratio(name, entrada)

    similares['similaridade'] = similares['first_name'].apply(calculate_similarity)

    # Pegando os mais similares
    busca = similares[similares['similaridade'] == similares['similaridade'].max()][['classification', 'frequency_total']]

    # Classificando e agrupando para somar a maior frequência por gênero
    agrupado = busca.groupby('classification')['frequency_total'].sum().reset_index()

    # Determinando o gênero com maior frequência
    genero_mais_frequente = agrupado.loc[agrupado['frequency_total'].idxmax(), 'classification']
    if genero_mais_frequente == 'M':
        genero_mais_frequente = 'Masculino'
    elif genero_mais_frequente == 'F':
        genero_mais_frequente = 'Feminino'

    print(f'O gênero de {nome} é: {genero_mais_frequente} com base na sua frequência.')

C:\Users\wesley\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
Downloading...
From: https://drive.google.com/uc?id=1_KXXdjcDxVs7M3DN68NwVUxMaN99OaRq
To: C:\Users\wesley\Downloads\ML-genero.h5
100%|██████████████████████████████████████████████████████████████████████████████| 55.3k/55.3k [00:00<00:00, 670kB/s]


In [6]:
nome = input("Digite um nome: ")
qual_genero(nome)

Digite um nome:  Epfanio


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Para o nome Epfanio, o modelo sugere que seja masculino.
